### Limpieza de la base de datos
Proyecto:  <a href="https://github.com/Juansemv/MCPP_juan.munoz/tree/master/Proyecto%20final">Análisis tutelar en temas de salud 1992 - 2019</a>

Autor: <a href="https://www.linkedin.com/in/sebastianmunozv/">Juan Sebastian Muñoz Vargas</a>


Objetivo: Limpiar la información de las tutelas extraídas de la 
<a href="http://www.corteconstitucional.gov.co/relatoria/radicador/buscar.php?ponente=&demandado=&Sentencia=&Tipo=Sentencias&busqueda=salud&conector=AND&segundotema=&anios=Todos&pg=0&vs=0&accion=Buscar">página de la Corte Constitucional</a>

Fecha de creación: Noviembre de 2019

In [1]:
import nltk                     #Para Análisis del texto
from nltk import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import timeit                   #Para calcular tiempos de ejecución del código
import re                       #Para uso de expresiones regulares
import matplotlib.pyplot as plt #Para realizar los gráficos
import requests                 #Para hacer las solicitudes del codigo fuente de las páginas
from bs4 import BeautifulSoup   #Para transformar el código extraído a formato para poder ser usado en Python
import pandas as pd             #Decarga de pandas para uso de bases de datos
plt.rcParams["figure.figsize"] = [18.0, 8.0]

In [2]:
#Se carga la base de datos
Database_tutelas = pd.read_pickle('DB_Tutelas.pkl')
len(Database_tutelas)

5837

In [3]:
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,Tema
0,1,2130,1992,08,11,http://www.corteconstitucional.gov.co/relatori...,T-484/92,FABIO MORÓN DÍAZ,ALONSO MUÑOZ,.,DERECHO A LA SALUD. DERECHO A LA VIDA. SIDA. ...
1,2,2047,1992,08,11,http://www.corteconstitucional.gov.co/relatori...,T-487/92,ALEJANDRO MARTÍNEZ CABALLERO,DANILO MOLINA,.,DERECHO A LA SALUD. LIBERTAD DE LOCOMOCION. L...
2,3,2193,1992,08,13,http://www.corteconstitucional.gov.co/relatori...,T-491/92,EDUARDO CIFUENTES MUÑOZ,ROSALBA DUQUE,.,DERECHO A LA SEGURIDAD SOCIAL. DERECHO A LA S...
3,4,2359,1992,08,21,http://www.corteconstitucional.gov.co/relatori...,T-499/92,EDUARDO CIFUENTES MUÑOZ,NERY CHIQUIZA,.,DERECHO A LA SALUD. DIGNIDAD HUMANA. CAPRECOM...
4,5,2535,1992,08,28,http://www.corteconstitucional.gov.co/relatori...,T-505/92,EDUARDO CIFUENTES MUÑOZ,DIEGO SERNA,.,SERVICIO DE SALUD. DERECHOS DEL ENFERMO. SIDA...
5,6,2972,1992,09,19,http://www.corteconstitucional.gov.co/relatori...,T-522/92,ALEJANDRO MARTÍNEZ CABALLERO,BALDOYNO ASPRILLA,.,DERECHOS DE LOS INTERNOS. DERECHO A LA SALUD....
6,7,2610,1992,09,23,http://www.corteconstitucional.gov.co/relatori...,T-536/92,SIMON RODRIGUEZ RODRIGUEZ,OLINDA BARRAGAN Y OTRA,.,DERECHO AL AMBIENTE SANO. DERECHO A LA SALUD....
7,8,3115,1992,09,24,http://www.corteconstitucional.gov.co/relatori...,T-534/92,CIRO ANGARITA BARON,JORGE A MORENO,.,DERECHO A LA SALUD. SERVICIO MILITAR. DERECHO...
8,9,3479,1992,10,02,http://www.corteconstitucional.gov.co/relatori...,T-548/92,CIRO ANGARITA BARON,FELIX ROJAS,.,DERECHO A LA SALUD. DERECHOS DE LOS PACIENTES...
9,10,D-039,1992,10,20,http://www.corteconstitucional.gov.co/relatori...,C-560/92,SIMON RODRIGUEZ RODRIGUEZ,ABDON ORTIZ,.,DEC. 1272/90. ARTS. 3 LIT p) Y 7 LIT ll). REF...


In [4]:
#Limpieza de links y formatos especiales dentro de 'tema'
def clean_color(text):
    return re.sub('<span style="color:black;background:#FF8000">.*?</span>', 'salud', text)
Database_tutelas['Tema'] = Database_tutelas['Tema'].apply(clean_color)
def clean_links(text):
    return re.sub('<a.*?a>', '', text)
Database_tutelas['Tema'] = Database_tutelas['Tema'].apply(clean_links)
Database_tutelas['Tema'][3500]

'   DERECHOS AL TRABAJO, IGUALDAD, SALUD Y SEGURIDAD SOCIAL QUE EL ACTOR CONSIDERA CONCULCADOS POR LA ACCIONADA CON LA EMISION A SU NOMBRE DE UNA SEGUNDA CEDULA CON DATOS QUE NO LE CORRESPONDEN Y LA CANCELACION DE LA PRIMERA QUE SI LO IDENTIFICABA CORRECTAMENTE. SEGUN MANIFIESTA,  ELLO LE HA CAUSADO MULTIPLES CONFLICTOS QUE HAN AFECTADO LOS DERECHOS DE LA REFERENCIA.'

In [5]:
#Defino puntuación para eliminarla
puntuacion = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–¡¿”“•\r´–¡¿”“•\r´º'
puntuacion

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–¡¿”“•\r´–¡¿”“•\r´º'

In [6]:
#Defino una función que elimine la puntuación
def no_punt(string):
    transtable = string.maketrans('', '', puntuacion)
    return string.translate(transtable)
Database_tutelas['Tutela_no_punt'] = Database_tutelas['Tema'].apply(no_punt)

La columna 'Tema' con puntuación VS. 'Tema' sin puntuación:

In [7]:
#'Tema' con puntuación
print(Database_tutelas['Tema'][5800])

 DERECHO AL GOCE EFECTIVO DE LA MESADA PENSIONAL, ANTE IMPOSIBILIDAD FÍSICA Y/O MENTAL DEL TITULAR DEL DERECHO PARA RECLAMARLA 
 La accionante actúa como agente oficiosa de su cónyuge quien tiene más de 85 años de edad, padece demencia senil, alcoholismo crónico y otras patologías.  En la misma condición actúa en favor de un hijo mayor de edad diagnosticado con el síndrome de Gilles de la Tourette.  El esposo de la actora es beneficiario de una pensión de vejez y esta prestación es la única fuente de ingresos del núcleo familiar. La peticionaria es quien hace los retiros de las mesadas pensionales en la entidad bancaria accionada, a través de una tarjeta débito a nombre del pensionado, la cual se venció y debe ser renovada sólo por el titular de la cuenta de ahorros y a través de su presentación personal.  Este trámite es el que exige el banco y el que la accionante alega como imposible de cumplir, debido a las condiciones de


In [8]:
#'Tema' sin puntuación
print(Database_tutelas['Tutela_no_punt'][5800])

 DERECHO AL GOCE EFECTIVO DE LA MESADA PENSIONAL ANTE IMPOSIBILIDAD FÍSICA YO MENTAL DEL TITULAR DEL DERECHO PARA RECLAMARLA 
 La accionante actúa como agente oficiosa de su cónyuge quien tiene más de 85 años de edad padece demencia senil alcoholismo crónico y otras patologías  En la misma condición actúa en favor de un hijo mayor de edad diagnosticado con el síndrome de Gilles de la Tourette  El esposo de la actora es beneficiario de una pensión de vejez y esta prestación es la única fuente de ingresos del núcleo familiar La peticionaria es quien hace los retiros de las mesadas pensionales en la entidad bancaria accionada a través de una tarjeta débito a nombre del pensionado la cual se venció y debe ser renovada sólo por el titular de la cuenta de ahorros y a través de su presentación personal  Este trámite es el que exige el banco y el que la accionante alega como imposible de cumplir debido a las condiciones de


In [9]:
#Paso a minúsculas todas las letras
Database_tutelas['Tutela_no_punt'] = Database_tutelas['Tutela_no_punt'].apply(str.lower)
print(Database_tutelas['Tutela_no_punt'][5800])

 derecho al goce efectivo de la mesada pensional ante imposibilidad física yo mental del titular del derecho para reclamarla 
 la accionante actúa como agente oficiosa de su cónyuge quien tiene más de 85 años de edad padece demencia senil alcoholismo crónico y otras patologías  en la misma condición actúa en favor de un hijo mayor de edad diagnosticado con el síndrome de gilles de la tourette  el esposo de la actora es beneficiario de una pensión de vejez y esta prestación es la única fuente de ingresos del núcleo familiar la peticionaria es quien hace los retiros de las mesadas pensionales en la entidad bancaria accionada a través de una tarjeta débito a nombre del pensionado la cual se venció y debe ser renovada sólo por el titular de la cuenta de ahorros y a través de su presentación personal  este trámite es el que exige el banco y el que la accionante alega como imposible de cumplir debido a las condiciones de


In [10]:
# Se elimina la puntuación y se coloca en miníscula la columna demandado
Database_tutelas['Demandado_no_punt'] = Database_tutelas['Demandado'].apply(no_punt)
Database_tutelas['Demandado_no_punt'] = Database_tutelas['Demandado_no_punt'].apply(str.lower)
print(Database_tutelas['Demandado_no_punt'][5808])

 inpec y otros


In [11]:
#Deja solo al magistrado ponente:
def clean_magistrado(text):
    return re.sub('<.*?   ', '', text)
Database_tutelas['Magistrado_Ponente'] = Database_tutelas['Magistrado_Ponente'].apply(clean_magistrado)
Database_tutelas['Magistrado_Ponente'][0]

'FABIO MORÓN DÍAZ                   '

In [12]:
#Se pone en formato numérico las columnas 'Año', 'Mes' y 'Día'
Database_tutelas['Año'] = pd.to_numeric(Database_tutelas['Año'], errors = "coerce")
Database_tutelas['Mes'] = pd.to_numeric(Database_tutelas['Mes'], errors = "coerce")
Database_tutelas['Día'] = pd.to_numeric(Database_tutelas['Día'], errors = "coerce")

In [13]:
# Añadir la columna de fecha en formato datetime
Database_tutelas['Fecha'] = pd.to_datetime(Database_tutelas.Año * 10000 +\
                                           Database_tutelas.Mes * 100 + Database_tutelas.Día, format='%Y%m%d')

In [14]:
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,Tema,Tutela_no_punt,Demandado_no_punt,Fecha
0,1,2130,1992,8,11,http://www.corteconstitucional.gov.co/relatori...,T-484/92,FABIO MORÓN DÍAZ,ALONSO MUÑOZ,.,DERECHO A LA SALUD. DERECHO A LA VIDA. SIDA. ...,derecho a la salud derecho a la vida sida iss...,,1992-08-11
1,2,2047,1992,8,11,http://www.corteconstitucional.gov.co/relatori...,T-487/92,ALEJANDRO MARTÍNEZ CABALLERO,DANILO MOLINA,.,DERECHO A LA SALUD. LIBERTAD DE LOCOMOCION. L...,derecho a la salud libertad de locomocion lib...,,1992-08-11
2,3,2193,1992,8,13,http://www.corteconstitucional.gov.co/relatori...,T-491/92,EDUARDO CIFUENTES MUÑOZ,ROSALBA DUQUE,.,DERECHO A LA SEGURIDAD SOCIAL. DERECHO A LA S...,derecho a la seguridad social derecho a la sa...,,1992-08-13
3,4,2359,1992,8,21,http://www.corteconstitucional.gov.co/relatori...,T-499/92,EDUARDO CIFUENTES MUÑOZ,NERY CHIQUIZA,.,DERECHO A LA SALUD. DIGNIDAD HUMANA. CAPRECOM...,derecho a la salud dignidad humana caprecom c...,,1992-08-21
4,5,2535,1992,8,28,http://www.corteconstitucional.gov.co/relatori...,T-505/92,EDUARDO CIFUENTES MUÑOZ,DIEGO SERNA,.,SERVICIO DE SALUD. DERECHOS DEL ENFERMO. SIDA...,servicio de salud derechos del enfermo sida c...,,1992-08-28
5,6,2972,1992,9,19,http://www.corteconstitucional.gov.co/relatori...,T-522/92,ALEJANDRO MARTÍNEZ CABALLERO,BALDOYNO ASPRILLA,.,DERECHOS DE LOS INTERNOS. DERECHO A LA SALUD....,derechos de los internos derecho a la salud c...,,1992-09-19
6,7,2610,1992,9,23,http://www.corteconstitucional.gov.co/relatori...,T-536/92,SIMON RODRIGUEZ RODRIGUEZ,OLINDA BARRAGAN Y OTRA,.,DERECHO AL AMBIENTE SANO. DERECHO A LA SALUD....,derecho al ambiente sano derecho a la salud m...,,1992-09-23
7,8,3115,1992,9,24,http://www.corteconstitucional.gov.co/relatori...,T-534/92,CIRO ANGARITA BARON,JORGE A MORENO,.,DERECHO A LA SALUD. SERVICIO MILITAR. DERECHO...,derecho a la salud servicio militar derecho a...,,1992-09-24
8,9,3479,1992,10,2,http://www.corteconstitucional.gov.co/relatori...,T-548/92,CIRO ANGARITA BARON,FELIX ROJAS,.,DERECHO A LA SALUD. DERECHOS DE LOS PACIENTES...,derecho a la salud derechos de los pacientes ...,,1992-10-02
9,10,D-039,1992,10,20,http://www.corteconstitucional.gov.co/relatori...,C-560/92,SIMON RODRIGUEZ RODRIGUEZ,ABDON ORTIZ,.,DEC. 1272/90. ARTS. 3 LIT p) Y 7 LIT ll). REF...,dec 127290 arts 3 lit p y 7 lit ll reforma de...,,1992-10-20


In [15]:
#Quitar el doble espacio
def clean_spaces(text):
    return re.sub(' {2,}', '', text)
Database_tutelas['Magistrado_Ponente'] = Database_tutelas['Magistrado_Ponente'].apply(clean_spaces)

In [16]:
#Quitar puntos y comas, especialmente para que palabras como E.P.S. queden 'EPS'
#Además, se eliminan para el futuro análisis
def clean_dot(text):
    return re.sub('\.', '', text)
Database_tutelas['Demandado'] = Database_tutelas['Demandado'].apply(clean_dot)
def clean_comma(text):
    return re.sub(',', ' ', text)
Database_tutelas['Demandado'] = Database_tutelas['Demandado'].apply(clean_comma)

In [17]:
Database_tutelas['Demandado'][600:1000]

600                                                     
601                                                     
602     ESE HOSPITAL LA MARIA DE MEDELLIN Y EL SISBEN...
603                                                     
604                             ALVARO URREA BOTERO VS  
605     INSTITUTO DE SEGURO SOCIAL SECCIONAL CUNDINAM...
606                                                     
607     INSTITUTO DE SUGUROS SOCIALES Y OTRO (VALLE) ...
608                                   SALUDCOOP EPS VS  
609                               SALUD TOTAL - EPS VS  
610           INSTITUTO DE SEGUROS SOCIALES (VALLE) VS  
611                                POLICIA NACIONAL VS  
612                             ACERIAS PAZ DEL RIO VS  
613                                   CAFESALUD EPS VS  
614                                   CAFESALUD EPS VS  
615     INSTITUTO DE SEGUROS SOCIALES DE BARRANQUILLA...
616                   INSTITUTO DE SEGUROS SOCIALES VS  
617                            

In [18]:
#Revisar que no hayan duplicados
Database_tutelas.drop_duplicates(subset = ('Sentencia'), inplace = True)
len(Database_tutelas)

5834

5837 - 3834 = 3

Por lo tanto, hay 3 duplicados

In [19]:
# Organizo por fecha
Database_tutelas.sort_values('Fecha', inplace = True)

In [20]:
# Reinicio los índices
Database_tutelas.reset_index(inplace = True, drop = True)
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,Tema,Tutela_no_punt,Demandado_no_punt,Fecha
0,1,2130,1992,8,11,http://www.corteconstitucional.gov.co/relatori...,T-484/92,FABIO MORÓN DÍAZ,ALONSO MUÑOZ,,DERECHO A LA SALUD. DERECHO A LA VIDA. SIDA. ...,derecho a la salud derecho a la vida sida iss...,,1992-08-11
1,2,2047,1992,8,11,http://www.corteconstitucional.gov.co/relatori...,T-487/92,ALEJANDRO MARTÍNEZ CABALLERO,DANILO MOLINA,,DERECHO A LA SALUD. LIBERTAD DE LOCOMOCION. L...,derecho a la salud libertad de locomocion lib...,,1992-08-11
2,3,2193,1992,8,13,http://www.corteconstitucional.gov.co/relatori...,T-491/92,EDUARDO CIFUENTES MUÑOZ,ROSALBA DUQUE,,DERECHO A LA SEGURIDAD SOCIAL. DERECHO A LA S...,derecho a la seguridad social derecho a la sa...,,1992-08-13
3,4,2359,1992,8,21,http://www.corteconstitucional.gov.co/relatori...,T-499/92,EDUARDO CIFUENTES MUÑOZ,NERY CHIQUIZA,,DERECHO A LA SALUD. DIGNIDAD HUMANA. CAPRECOM...,derecho a la salud dignidad humana caprecom c...,,1992-08-21
4,5,2535,1992,8,28,http://www.corteconstitucional.gov.co/relatori...,T-505/92,EDUARDO CIFUENTES MUÑOZ,DIEGO SERNA,,SERVICIO DE SALUD. DERECHOS DEL ENFERMO. SIDA...,servicio de salud derechos del enfermo sida c...,,1992-08-28
5,6,2972,1992,9,19,http://www.corteconstitucional.gov.co/relatori...,T-522/92,ALEJANDRO MARTÍNEZ CABALLERO,BALDOYNO ASPRILLA,,DERECHOS DE LOS INTERNOS. DERECHO A LA SALUD....,derechos de los internos derecho a la salud c...,,1992-09-19
6,7,2610,1992,9,23,http://www.corteconstitucional.gov.co/relatori...,T-536/92,SIMON RODRIGUEZ RODRIGUEZ,OLINDA BARRAGAN Y OTRA,,DERECHO AL AMBIENTE SANO. DERECHO A LA SALUD....,derecho al ambiente sano derecho a la salud m...,,1992-09-23
7,8,3115,1992,9,24,http://www.corteconstitucional.gov.co/relatori...,T-534/92,CIRO ANGARITA BARON,JORGE A MORENO,,DERECHO A LA SALUD. SERVICIO MILITAR. DERECHO...,derecho a la salud servicio militar derecho a...,,1992-09-24
8,9,3479,1992,10,2,http://www.corteconstitucional.gov.co/relatori...,T-548/92,CIRO ANGARITA BARON,FELIX ROJAS,,DERECHO A LA SALUD. DERECHOS DE LOS PACIENTES...,derecho a la salud derechos de los pacientes ...,,1992-10-02
9,10,D-039,1992,10,20,http://www.corteconstitucional.gov.co/relatori...,C-560/92,SIMON RODRIGUEZ RODRIGUEZ,ABDON ORTIZ,,DEC. 1272/90. ARTS. 3 LIT p) Y 7 LIT ll). REF...,dec 127290 arts 3 lit p y 7 lit ll reforma de...,,1992-10-20


In [21]:
#Creo dos columna de tokens, una para las tutelas y otra para los demandados
Database_tutelas['Tutela_tkn'] = Database_tutelas['Tutela_no_punt'].apply(word_tokenize)
Database_tutelas['Demandado_tkn'] = Database_tutelas['Demandado_no_punt'].apply(word_tokenize)

In [22]:
# Adquiero las stopwords en español y las elimino
stopwords_esp = stopwords.words('spanish')
def no_stopwords(tokens):
    return [p for p in tokens if p not in stopwords_esp]

In [23]:
#Agrego una columna para la tutela y el demandado sin stopwords
Database_tutelas['Tutela_nsw'] = Database_tutelas['Tutela_tkn'].apply(no_stopwords)
Database_tutelas['Demandado_nsw'] = Database_tutelas['Demandado_tkn'].apply(no_stopwords)

In [24]:
#Paso a texto los tokens de los demandados y de las tutelas
Database_tutelas['Tutela_text'] = Database_tutelas['Tutela_nsw'].apply(nltk.Text)
Database_tutelas['Demandado_text'] = Database_tutelas['Demandado_nsw'].apply(nltk.Text)

In [25]:
Database_tutelas.loc[5000:5100]

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,Tema,Tutela_no_punt,Demandado_no_punt,Fecha,Tutela_tkn,Demandado_tkn,Tutela_nsw,Demandado_nsw,Tutela_text,Demandado_text
5000,5004,T-4032900,2014,1,27,http://www.corteconstitucional.gov.co/relatori...,T-028/14,MARÍA VICTORIA CALLE CORREA,YOSIRA COROMOTO BERMUDEZ,AGUAS DE LA PENINSULA SA ESP,Se le endilga a la empresa Aguas de la Pení...,se le endilga a la empresa aguas de la pení...,aguas de la peninsula sa esp,2014-01-27,"[se, le, endilga, a, la, empresa, aguas, de, l...","[aguas, de, la, peninsula, sa, esp]","[endilga, empresa, aguas, península, sa, munic...","[aguas, peninsula, sa, esp]","(endilga, empresa, aguas, península, sa, munic...","(aguas, peninsula, sa, esp)"
5001,5006,LAT-411,2014,1,29,http://www.corteconstitucional.gov.co/relatori...,C-036/14,LUIS ERNESTO VARGAS SILVA,REVISION CONSTITUCIONAL DE LA LEY 1623 DE 2013...,,Revisión Constitucional de la Ley 1623 de 2...,revisión constitucional de la ley 1623 de 2...,,2014-01-29,"[revisión, constitucional, de, la, ley, 1623, ...",[],"[revisión, constitucional, ley, 1623, 2013, me...",[],"(revisión, constitucional, ley, 1623, 2013, me...",()
5002,5007,T-4042165,2014,1,31,http://www.corteconstitucional.gov.co/relatori...,T-042/14,LUIS ERNESTO VARGAS SILVA,LEVIS MARINA POTE DE LAS SALAS,SECRETARIA DE EDUCACION DEPARTAMENTAL DE BOLIVAR,Aduce la peticionaria que sus derechos fund...,aduce la peticionaria que sus derechos fund...,secretaria de educacion departamental de bolivar,2014-01-31,"[aduce, la, peticionaria, que, sus, derechos, ...","[secretaria, de, educacion, departamental, de,...","[aduce, peticionaria, derechos, fundamentales,...","[secretaria, educacion, departamental, bolivar]","(aduce, peticionaria, derechos, fundamentales,...","(secretaria, educacion, departamental, bolivar)"
5003,5008,T-4052900 Y OTRO ACUMULADOS,2014,2,3,http://www.corteconstitucional.gov.co/relatori...,T-048/14,GABRIEL EDUARDO MENDOZA MARTELO,RUTH EMENITH DUARTE BAUTISTA Y OTRO,NUEVA EPS Y OTRA,"Salud, vida, dignidad humana. Se acumulan ...",salud vida dignidad humana se acumulan exp...,nueva eps y otra,2014-02-03,"[salud, vida, dignidad, humana, se, acumulan, ...","[nueva, eps, y, otra]","[salud, vida, dignidad, humana, acumulan, expe...","[nueva, eps]","(salud, vida, dignidad, humana, acumulan, expe...","(nueva, eps)"
5004,5009,T-4061327,2014,2,3,http://www.corteconstitucional.gov.co/relatori...,T-052/14,ALBERTO ROJAS RÍOS,JUAN MONTOYA RAMIREZ,UNIDAD ADMINISTRATIVA ESPECIAL DE GESTION PEN...,El accionante solicita que se le protejan s...,el accionante solicita que se le protejan s...,unidad administrativa especial de gestion pen...,2014-02-03,"[el, accionante, solicita, que, se, le, protej...","[unidad, administrativa, especial, de, gestion...","[accionante, solicita, protejan, derechos, fun...","[unidad, administrativa, especial, gestion, pe...","(accionante, solicita, protejan, derechos, fun...","(unidad, administrativa, especial, gestion, pe..."
5005,5010,T-4053884 Y OTROS ACUMULADOS,2014,2,3,http://www.corteconstitucional.gov.co/relatori...,T-054/14,ALBERTO ROJAS RÍOS,YULIS ELENA NAVARRO JARAMILLO Y OTROS,EPS CAPRECOM Y OTRAS,"Salud, vida en condiciones dignas. Se acum...",salud vida en condiciones dignas se acumul...,eps caprecom y otras,2014-02-03,"[salud, vida, en, condiciones, dignas, se, acu...","[eps, caprecom, y, otras]","[salud, vida, condiciones, dignas, acumulan, e...","[eps, caprecom]","(salud, vida, condiciones, dignas, acumulan, e...","(eps, caprecom)"
5006,5011,T-4054261 Y OTRO ACUMULADOS,2014,2,3,http://www.corteconstitucional.gov.co/relatori...,T-061/14,NILSON PINILLA PINILLA,BLANCA ALICIA CARVAJAL DIAZ Y OTROS,SALUD TOTAL EPS Y OTROS,"Salud, integridad física, vida digna. Se a...",salud integridad física vida digna se acum...,salud total eps y otros,2014-02-03,"[salud, integridad, física, vida, digna, se, a...","[salud, total, eps, y, otros]","[salud, integridad, física, vida, digna, acu

In [26]:
Database_tutelas.to_pickle('DbTutelas.pkl')

---